In [ ]:
import numpy as np

In [ ]:
Z = np.arange(9).reshape(3,3).astype(np.int16)

In [ ]:
print("#bytes    :", Z.itemsize)
print("shape     :", Z.shape)
print("dimensions:", Z.ndim)
print("type.     :", Z.dtype)

In [ ]:
np.arange(9).reshape(3,3).dtype

the strides of the array that define the **number of bytes to step in each dimension** when traversing the array

In [ ]:
assert Z.strides == (Z.shape[1]*Z.itemsize, Z.itemsize)
print("itemsize :", Z.itemsize, "bytes")
print("strides  :", Z.strides, "bytes")

In [ ]:
Z

In [ ]:
index = 1,1
Z[index]

In [ ]:
Z[index].tobytes()

In [ ]:
offset = 0
for d in range(Z.ndim):
    offset += Z.strides[d]*index[d] 

print("bytes offset", offset)
print("index offset", offset//Z.itemsize)
print("value:")
print(Z.tobytes()[offset: offset+Z.itemsize])
print(Z[index])
print(Z.flatten()[offset//Z.itemsize])

## Views and Copies

In [ ]:
Z = np.zeros(9)

In [ ]:
Z1= Z[:3]
Z2=Z[[0,1,2]]
index = [0,1,2]
Z3 = Z[index]

In [ ]:
print(Z1.base is Z)
print(Z2.base in Z)
print(Z3.base is Z)
print(Z[index].base is Z)

In [ ]:
Z[index] = 1
print(Z)

In [ ]:
print(Z1)
print(Z2)
print(Z3)

In [ ]:
Z = np.random.uniform(0,1,(5,5))
Z[:3, :].base is Z

In [ ]:
print(Z.ravel().base is Z)
print(Z.flatten().base is Z)

In [ ]:
Z.base is None

## Temporary copies

In [ ]:
from timeit import timeit
import numpy as np

In [ ]:
X = np.ones(1000000000, dtype=np.int)
Y = np.ones(1000000000, dtype=np.int)

### Conclusion Exos

In [2]:
import numpy as np
Z1 = np.arange(10)
Z2 = Z1[1:-1:2]

In [3]:
print(Z1)
print(Z2)

[0 1 2 3 4 5 6 7 8 9]
[1 3 5 7]


In [4]:
Z2.base is Z1

True

Therefore ``Z2`` can be expressed as ``Z1[start:stop:step]``

In [7]:
print(Z1.strides)
print(Z2.strides) # notice that this is double because Z2.base is Z1
print(np.array([1,3,5,7]).strides)

(8,)
(16,)
(8,)


In [9]:
print(Z1.itemsize, Z1.dtype)
print(Z2.itemsize, Z2.dtype)

8 int64
8 int64


In [31]:
offset_start = np.byte_bounds(Z2)[0] - np.byte_bounds(Z1)[0] # bytes
offset_stop = np.byte_bounds(Z2)[1] - np.byte_bounds(Z1)[1] # bytes

start,stop = offset_start//Z1.itemsize, offset_stop//Z1.itemsize
step = Z2.strides[0] // Z1.strides[0] 

In [32]:
print(offset_start, offset_stop)
print(offset_start//Z1.itemsize, Z1.size + offset_stop//Z1.itemsize)
print(step)

8 -16
1 8
2


In [39]:
Z1[start:stop:step] == Z2

array([ True,  True,  True,  True])

In [40]:
np.allclose(Z1[start:stop:step], Z2)

True

### Exo

- negative steps
- multi-dimensional arrays

In [48]:
import numpy as np
Z1 = np.arange(10)
Z2 = Z1[5::-2]

In [49]:
print(Z1)
print(Z2)

[0 1 2 3 4 5 6 7 8 9]
[5 3 1]


In [60]:
print(Z2.base is Z1)
print(Z2.strides)
print(Z2.itemsize, Z2.dtype)

True
(-16,)
8 int64


In [55]:
bb_z1 = np.byte_bounds(Z1)
bb_z2 = np.byte_bounds(Z2)

print(bb_z2, bb_z2[1]-bb_z2[0]) # bounds does not account for the order, just locates bounds

True
(-16,)
(140392102629688, 140392102629728) 40


In [51]:
Z2.strides

(-16,)

In [76]:
stop = (bb_z2[0] - bb_z1[0])//Z1.itemsize -1
start = (bb_z2[1] - bb_z1[1])//Z1.itemsize -1
step =  Z2.strides[0]//Z1.strides[0]
print(start, stop, step)
index = slice(start, stop, step)

-5 0 -2


In [77]:
Z1[index]

array([5, 3, 1])

In [69]:
Z2.ndim

1

In [ ]:
def find_index(base:np.ndarray, view: np.ndarray):

    assert view.base is base

    # bounding box in bytes (global)
    view_bottom, biew_top = np.byte_bounds(view)
    base_bottom, base_top = np.byte_bounds(base)
    assert base_bottom <= view_bottom
    assert view_top <= base_top

    stop = (bb_z2[0] - bb_z1[0])//Z1.itemsize -1
    start = (bb_z2[1] - bb_z1[1])//Z1.itemsize -1

    # strides are in bytes
    steps = [ view.strides[n] // base.strides[n] for n in range(base.ndim) ]

    assert all(step!=0)

    if step>0:


In [78]:
base = np.arange(8*8).reshape(8,8)
base

array([[ 0,  1,  2,  3,  4,  5,  6,  7],
       [ 8,  9, 10, 11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29, 30, 31],
       [32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47],
       [48, 49, 50, 51, 52, 53, 54, 55],
       [56, 57, 58, 59, 60, 61, 62, 63]])

In [79]:
view = base[1:-1:2, 5::-2]
view

array([[13, 11,  9],
       [29, 27, 25],
       [45, 43, 41]])

In [72]:
base.strides

(64, 8)

In [80]:
view.strides

(128, -16)

In [81]:
[view.strides[n] // base.strides[n] for n in range(base.ndim) ]

[2, -2]